In [4]:
import pandas as pd
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report,make_scorer,recall_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.dummy import DummyClassifier


In [5]:
df = pd.read_csv('https://raw.githubusercontent.com/joshuasir/datasets/main/stock-market-train-clean.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/joshuasir/datasets/main/stock-market-test-clean.csv')

In [6]:
X = df.copy()
# X = df.drop(columns=['Date','Adj Close','CloseUSD','Volume','RSI14','Open','Year'],axis=1).copy()
y = X.pop('Target')
X_valid = df_test.copy()
y_valid = X_valid.pop('Target')

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,stratify=y,random_state=42)

In [7]:
y.value_counts()

0    68651
1    14020
Name: Target, dtype: int64

we will be using cross validation and recall since our data is unbalance

## Baseline Model

In [8]:
model = DummyClassifier(strategy='stratified')


scoring = {'Accuracy': 'accuracy',
           'Precision': 'precision_macro',
           'Recall': 'recall_macro',
           'F1':'f1_macro',
    }


scores = cross_validate(model, X_train, y_train,
                              cv=5,
                              scoring=scoring)

print("Precision scores:\n", scores['test_Precision'].mean())
print("Recall scores:\n", scores['test_Recall'].mean())
print("F1 scores:\n", scores['test_F1'].mean())

# print('Dummy Classifier :', model.score(y_test,pred))
# print(classification_report(y_test,pred))

Precision scores:
 0.4986067305989758
Recall scores:
 0.49860828773577026
F1 scores:
 0.49860257555400417


## Comparing Model

In [10]:

def train(classifier,name='',graph=False):
    
    scoring = {'Accuracy': 'accuracy',
           'Precision': 'precision_macro',
           'Recall': 'recall_macro',
           'F1':'f1_macro',
           'ROC_AUC'  : 'roc_auc'
    }
    score_dict = cross_validate(classifier, X_test, y_test,
                              cv=5,
                              scoring=scoring)
    
    scores = pd.DataFrame(columns=['Model','Precision','Recall','F1','ROC AUC'])
    scores = scores.append(pd.Series({ 
                                      'Model' : name,
                                      # 'Accuracy' : score_dict['test_Accuracy'].mean(),
                                      'Precision' : score_dict['test_Precision'].mean(),
                                      'Recall' : score_dict['test_Recall'].mean(),
                                      'F1' : score_dict['test_F1'].mean(),
                                      'ROC AUC' : score_dict['test_ROC_AUC'].mean()
    }),ignore_index=True)
 
    return scores

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, make_scorer


models =  { "Decision Tree" : DecisionTreeClassifier(),
            "Random Forest" : RandomForestClassifier(),
            "XGBoost" : XGBClassifier(), 
            "LogisticRegression" : LogisticRegression(max_iter=1200), 
            "CatBoost" :CatBoostClassifier(verbose=False) ,
            'LGBM' : LGBMClassifier()
            
            }

scores = pd.DataFrame(columns=['Model','Precision','Recall','F1','ROC AUC'])
# probability = {}

for model in models:
  scores = scores.append(train(models[model],name=model))

In [13]:
scores.sort_values('Recall',ascending=False).reset_index(drop=True)

,Model,Precision,Recall,F1,ROC AUC
0,LogisticRegression,0.776489,0.780509,0.778000,0.928984
1,LGBM,0.820723,0.750410,0.778058,0.937964
2,CatBoost,0.810967,0.749652,0.774454,0.936750
3,Decision Tree,0.745274,0.746653,0.745869,0.746653
4,XGBoost,0.826467,0.744556,0.775617,0.937831
5,Random Forest,0.782909,0.740587,0.758611,0.929290


## Model Tuning

In [16]:
dict_score = cross_validate(LogisticRegression(max_iter=2200), X, y,
                              cv=15,
                              scoring='recall_macro')

In [20]:
dict_score['test_score'].mean()

0.7283244077244543

## Model Train